## 安裝套件

In [1]:
!pip install jieba
!pip install emoji langdetect
!pip install datasets
!pip install lingua-language-detector

In [2]:
import sys
!{sys.executable} -m pip install jieba emoji langdetect pytz torch lingua-language-detector datasets openpyxl


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip


## 引入套件

In [8]:
pip install transformers

  Using cached regex-2024.11.6-cp311-cp311-macosx_11_0_arm64.whl.metadata (40 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 17.2 MB/s eta 0:00:00 0:00:01
Using cached regex-2024.11.6-cp311-cp311-macosx_11_0_arm64.whl (284 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 18.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import numpy as np

import re
import emoji
import langdetect
from langdetect import detect
from lingua import LanguageDetectorBuilder, Language, IsoCode639_1
from datetime import datetime
import pytz
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModel
import torch

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 設定好路徑 (後面都是使用相對路徑)
base_path = '/content/drive/My Drive/SMA'

Mounted at /content/drive


In [2]:
# 讀取資料（請確認你的 Excel 路徑）
# df = pd.read_excel(base_path+"/threads.xlsx")
df = pd.read_excel("threads.xlsx", engine='openpyxl')
df

,author,post_time,topic,time_info,content,has_photo,has_video,like_count,reply_count,repost_count,share_count,view_count,followers_count,post_url,scrape_time
0,ayofvr,2025-04-29T10:58:39.000Z,NaN,3小時,Thank you God for another day.,N,N,190,3,23,NaN,"3,197",141073,https://www.threads.net/@ayofvr/post/DJBymf8uTrK,2025-04-29T22:27:40.176749
1,NaN,2025-04-29T12:33:44.000Z,NaN,1小時,百達翡麗？ 沒有下限的網路病態！,Y,N,196,16,NaN,6,3 萬,77683,https://www.threads.net/@ban.mei.onnnnni/post/...,2025-04-29T22:27:54.964788
2,ribboworld2021,2025-04-29T04:39:46.000Z,小一日常,9小時,考完期中考，成績都還沒出來，小一女兒就自信的對我說：「我真羨慕妳生了一個天才！」,N,N,75,6,NaN,NaN,"4,559",99,https://www.threads.net/@ribboworld2021/post/D...,2025-04-29T22:28:09.873641
3,ayofvr,2025-04-29T11:25:22.000Z,NaN,3小時,Just be strong. Confident. Hopeful. Intellectu...,N,N,83,5,3,1,"1,967",141093,https://www.threads.net/@ayofvr/post/DJB1qP5OmzP,2025-04-29T22:28:24.726576
4,jose_ykc,2025-04-29T10:36:19.000Z,輔仁大學,3小時,日文輔系老師上課內容之一AiScReam 歌詞導讀,Y,Y,"4,334",55,513,"2,460",10 萬,65,https://www.threads.net/@jose_ykc/post/DJBvpGI...,2025-04-29T22:28:39.393706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3096,leighton.williams,2025年04月29日 07:30,NaN,8小時,First tasting in California,N,N,7,NaN,NaN,0,726,30,https://www.threads.net/@leighton.williams/pos...,2025-04-29T16:23:40.328046
3097,cape__man,2025年04月29日 06:31,NaN,9小時,I hoped it would have been better.,N,N,2,NaN,NaN,0,"1,291",114,https://www.threads.net/@cape__man/post/DJAdFd...,2025-04-29T16:23:55.063517
3098,simimoonlight,2025年04月29日 06:26,NaN,9小時,I can’t wait to watch Beyoncé on TikTok tonigh...,N,N,"2,656",29,280,47,5.6 萬,65732,https://www.threads.net/@simimoonlight/post/DJ...,2025-04-29T16:24:39.870994
3099,other98,2025年04月29日 12:31,NaN,3小時,"Tonight, Canada just proved that they have a h...",N,N,1.0 萬,214,214,22,14 萬,217182,https://www.threads.net/@other98/post/DJBGV3NxiX_,2025-04-29T16:24:54.669936


In [3]:
# === 語言偵測修正版===
lingua_detector = LanguageDetectorBuilder.from_all_languages().with_preloaded_language_models().build()
lingua_available = True
def detect_lang_with_preprocessing_lingua(text):
    original_text = text

    # 若是 NaN 或空字串就回傳 "unknown"
    if pd.isna(text):
        return "unknown"
    text = str(text).strip()
    if not text:
        return "unknown"
    
    # 移除 URL、@標記、 #hashtag、emoji、多餘空白
    try:
      text_cleaned = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
      text_cleaned = re.sub(r'@\w+', '', text_cleaned)
      text_cleaned = re.sub(r'#\w+', '', text_cleaned)
      text_cleaned = emoji.replace_emoji(text_cleaned, replace='')
      text_cleaned = re.sub(r'\s+', ' ', text_cleaned).strip()
    except Exception as e:
      return "error_state_preprocessing"

    # 若這些清理完後變成空字串
    if not text_cleaned:
      return "empty_after_clean"

    # 若文字中超過 30% 是中文，就直接判定為 "Ch"（中文）
    try:
      chinese_chars = re.findall(r'[\u4e00-\u9fff]', text_cleaned)
      text_len = len(text_cleaned)
      ratio = len(chinese_chars) / max(text_len, 1)
      chinese_threshold = 0.3
      if ratio > chinese_threshold:
        return "Ch"

      # 呼叫 lingua 偵測語言
      detected_language = lingua_detector.detect_language_of(text_cleaned)

      # 若 lingua 判定是中文（'ZH'），則回傳 "Ch"，其餘語言以小寫的 ISO 639-1 回傳（如 en, ja, fr）
      # 若無法偵測出語言，回傳 "unknown"
      if detected_language is not None:
        iso_code = detected_language.iso_code_639_1.name
        if iso_code == 'ZH':
          return "Ch"
        else:
          return iso_code.lower()
      else:
        return "unknown"

    except Exception as e:
      return "unknown"

## 清洗數據V1

In [4]:
# === 數值欄位清洗（萬字、逗號格式處理）===
def parse_count(value):
    # 將文字數字（如 "1,234"、"2.5萬"）統一轉為整數（int）
    if pd.isna(value): return 0
    value = str(value).replace(",", "")
    # "萬" 的部分會乘上 10,000 做轉換
    # 無法處理的格式就回傳 0
    if "萬" in value:
        return int(float(value.replace("萬", "")) * 10000)
    try:
        return int(float(value))
    except:
        return 0

for col in ["like_count", "view_count", "share_count", "repost_count", "reply_count"]:
    df[col] = df[col].apply(parse_count)

# === 布林欄位處理 ===
# 將原始欄位（Y/N）轉換為 True/False
# 處理過程會去除空白、轉成大寫
df["has_photo"] = df["has_photo"].apply(lambda x: str(x).strip().upper() == "Y")
df["has_video"] = df["has_video"].apply(lambda x: str(x).strip().upper() == "Y")

# === emoji 萃取與統計 ===
# 檢查是否為文字型別，如果是文字，從中萃取出所有 emoji 字元並串接成字串回傳
def extract_emojis(text):
    if not isinstance(text, str): return ""
    return "".join([ch for ch in text if ch in emoji.EMOJI_DATA])

df["emojis"] = df["content"].apply(extract_emojis)
df["emoji_count"] = df["emojis"].apply(len)

# # === 語言偵測修正版===
# def detect_lang_custom(text):
#     try:
#         text = str(text)
#         chinese_chars = re.findall(r'[\u4e00-\u9fff]', text)
#         if len(chinese_chars) / max(len(text), 1) > 0.3:
#             return "Ch"
#         return detect(text)
#     except:
#         return "unknown"

# 使用先前定義好的語言偵測函數 detect_lang_with_preprocessing_lingua()，處理每篇文章的語言判定
df["lang"] = df["content"].apply(detect_lang_with_preprocessing_lingua)

# === scrape_time 處理（轉換時區 + 抽取星期與小時）===
# 將時間欄位轉為台北時區，額外抽出格式化後的時間字串、星期幾、小時(0–23）
df["scrape_time_origin"] = pd.to_datetime(df["scrape_time"], utc=True).dt.tz_convert("Asia/Taipei")
df["scrape_time"]  = df["scrape_time_origin"].dt.strftime("%Y年%m月%d日 %H:%M")
df["post_weekday"] = df["scrape_time_origin"].dt.day_name()
df["post_hour"] = df["scrape_time_origin"].dt.hour

# === 是否為高流量文章（破萬）===
# 超過等於 10,000 瀏覽為 1，其餘為 0
df["viral"] = (df["view_count"] >= 10000).astype(int)

# === 是否使用問號、驚嘆號 ===
df["has_question"] = df["content"].apply(lambda x: "？" in str(x) or "?" in str(x))
df["has_exclaim"] = df["content"].apply(lambda x: "！" in str(x) or "!" in str(x))

# === 儲存結果 ===
df.to_csv("threads_cleaned_v1.csv",encoding='utf_8_sig', index=False)
print("✅ 處理完成，已輸出 threads_cleaned_v1.csv")
display(df)


✅ 處理完成，已輸出 threads_cleaned_v1.csv


,author,post_time,topic,time_info,content,has_photo,has_video,like_count,reply_count,repost_count,...,scrape_time,emojis,emoji_count,lang,scrape_time_origin,post_weekday,post_hour,viral,has_question,has_exclaim
0,ayofvr,2025-04-29T10:58:39.000Z,NaN,3小時,Thank you God for another day.,False,False,190,3,23,...,2025年04月30日 06:27,,0,en,2025-04-30 06:27:40.176749+08:00,Wednesday,6,0,False,False
1,NaN,2025-04-29T12:33:44.000Z,NaN,1小時,百達翡麗？ 沒有下限的網路病態！,True,False,196,16,0,...,2025年04月30日 06:27,,0,Ch,2025-04-30 06:27:54.964788+08:00,Wednesday,6,1,True,True
2,ribboworld2021,2025-04-29T04:39:46.000Z,小一日常,9小時,考完期中考，成績都還沒出來，小一女兒就自信的對我說：「我真羨慕妳生了一個天才！」,False,False,75,6,0,...,2025年04月30日 06:28,,0,Ch,2025-04-30 06:28:09.873641+08:00,Wednesday,6,0,False,True
3,ayofvr,2025-04-29T11:25:22.000Z,NaN,3小時,Just be strong. Confident. Hopeful. Intellectu...,False,False,83,5,3,...,2025年04月30日 06:28,,0,en,2025-04-30 06:28:24.726576+08:00,Wednesday,6,0,False,False
4,jose_ykc,2025-04-29T10:36:19.000Z,輔仁大學,3小時,日文輔系老師上課內容之一AiScReam 歌詞導讀,True,True,4334,55,513,...,2025年04月30日 06:28,,0,Ch,2025-04-30 06:28:39.393706+08:00,Wednesday,6,1,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3096,leighton.williams,2025年04月29日 07:30,NaN,8小時,First tasting in California,False,False,7,0,0,...,2025年04月30日 00:23,,0,en,2025-04-30 00:23:40.328046+08:00,Wednesday,0,0,False,False
3097,cape__man,2025年04月29日 06:31,NaN,9小時,I hoped it would have been better.,False,False,2,0,0,...,2025年04月30日 00:23,,0,en,2025-04-30 00:23:55.063517+08:00,Wednesday,0,0,False,False
3098,simimoonlight,2025年04月29日 06:26,NaN,9小時,I can’t wait to watch Beyoncé on TikTok tonigh...,False,False,2656,29,280,...,2025年04月30日 00:24,🫶🏿🥹,3,en,2025-04-30 00:24:39.870994+08:00,Wednesday,0,1,False,False
3099,other98,2025年04月29日 12:31,NaN,3小時,"Tonight, Canada just proved that they have a h...",False,False,10000,214,214,...,2025年04月30日 00:24,🙌,1,en,2025-04-30 00:24:54.669936+08:00,Wednesday,0,1,False,True


## 清洗數據V2

In [5]:
# --- 文章長度 ---
df["content_length"] = df["content"].apply(lambda x: len(str(x)))

# --- 是否包含網址 ---
df["has_url"] = df["content"].apply(lambda x: "http" in str(x) or "www." in str(x))

# --- 是否包含 @標記他人 ---
df["has_mention"] = df["content"].apply(lambda x: "@" in str(x))

# --- 是否使用 Hashtag ---
df["has_hashtag"] = df["content"].apply(lambda x: "#" in str(x))

# 貼文主題字詞提取（可後續做 TF-IDF 或主題建模）
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=100, stop_words='english')
word_matrix = vectorizer.fit_transform(df['content'].astype(str))

# 將常見詞語提取出來
keywords = vectorizer.get_feature_names_out()

# 是否為深夜或白天貼文（時間段分類）
def time_period(hour):
    if 5 <= hour < 12:
        return "morning"
    elif 12 <= hour < 17:
        return "afternoon"
    elif 17 <= hour < 22:
        return "evening"
    else:
        return "night"

df["post_period"] = df["post_hour"].apply(time_period)

cols_to_show_first = ['author', 'content', 'content_length', 'lang', 'scrape_time', 'post_weekday', 'post_hour', 'post_period', 'viral']
df = df[cols_to_show_first + [col for col in df.columns if col not in cols_to_show_first]]
df.to_csv("threads_cleaned_v2.csv",encoding='utf_8_sig',index=False)
print("✅ 處理完成，已輸出 threads_cleaned_v2.csv")
display(df)

✅ 處理完成，已輸出 threads_cleaned_v2.csv


,author,content,content_length,lang,scrape_time,post_weekday,post_hour,post_period,viral,post_time,...,followers_count,post_url,emojis,emoji_count,scrape_time_origin,has_question,has_exclaim,has_url,has_mention,has_hashtag
0,ayofvr,Thank you God for another day.,30,en,2025年04月30日 06:27,Wednesday,6,morning,0,2025-04-29T10:58:39.000Z,...,141073,https://www.threads.net/@ayofvr/post/DJBymf8uTrK,,0,2025-04-30 06:27:40.176749+08:00,False,False,False,False,False
1,NaN,百達翡麗？ 沒有下限的網路病態！,16,Ch,2025年04月30日 06:27,Wednesday,6,morning,1,2025-04-29T12:33:44.000Z,...,77683,https://www.threads.net/@ban.mei.onnnnni/post/...,,0,2025-04-30 06:27:54.964788+08:00,True,True,False,False,False
2,ribboworld2021,考完期中考，成績都還沒出來，小一女兒就自信的對我說：「我真羨慕妳生了一個天才！」,40,Ch,2025年04月30日 06:28,Wednesday,6,morning,0,2025-04-29T04:39:46.000Z,...,99,https://www.threads.net/@ribboworld2021/post/D...,,0,2025-04-30 06:28:09.873641+08:00,False,True,False,False,False
3,ayofvr,Just be strong. Confident. Hopeful. Intellectu...,69,en,2025年04月30日 06:28,Wednesday,6,morning,0,2025-04-29T11:25:22.000Z,...,141093,https://www.threads.net/@ayofvr/post/DJB1qP5OmzP,,0,2025-04-30 06:28:24.726576+08:00,False,False,False,False,False
4,jose_ykc,日文輔系老師上課內容之一AiScReam 歌詞導讀,25,Ch,2025年04月30日 06:28,Wednesday,6,morning,1,2025-04-29T10:36:19.000Z,...,65,https://www.threads.net/@jose_ykc/post/DJBvpGI...,,0,2025-04-30 06:28:39.393706+08:00,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3096,leighton.williams,First tasting in California,27,en,2025年04月30日 00:23,Wednesday,0,night,0,2025年04月29日 07:30,...,30,https://www.threads.net/@leighton.williams/pos...,,0,2025-04-30 00:23:40.328046+08:00,False,False,False,False,False
3097,cape__man,I hoped it would have been better.,34,en,2025年04月30日 00:23,Wednesday,0,night,0,2025年04月29日 06:31,...,114,https://www.threads.net/@cape__man/post/DJAdFd...,,0,2025-04-30 00:23:55.063517+08:00,False,False,False,False,False
3098,simimoonlight,I can’t wait to watch Beyoncé on TikTok tonigh...,51,en,2025年04月30日 00:24,Wednesday,0,night,1,2025年04月29日 06:26,...,65732,https://www.threads.net/@simimoonlight/post/DJ...,🫶🏿🥹,3,2025-04-30 00:24:39.870994+08:00,False,False,False,False,False
3099,other98,"Tonight, Canada just proved that they have a h...",77,en,2025年04月30日 00:24,Wednesday,0,night,1,2025年04月29日 12:31,...,217182,https://www.threads.net/@other98/post/DJBGV3NxiX_,🙌,1,2025-04-30 00:24:54.669936+08:00,False,True,False,False,False


## Self-Attention 模組處理文字資料（測試中 因未寫完可先跳過）

In [8]:
def build_semantic_text(row):
    parts = []

    # 作者資訊加上前綴
    if pd.notna(row["author"]):
        parts.append(f"author_id: {row['author']}")

    # 主題分類加上前綴
    if pd.notna(row["topic"]):
        parts.append(f"topic: {row['topic']}")

    # Emoji 當作情感符號，合併成一串文字
    if pd.notna(row["emojis"]):
        emoji_text = " ".join(row["emojis"])
        parts.append(f"emojis: {emoji_text}")

    # 貼文正文
    if pd.notna(row["content"]):
        parts.append(f"content: {row['content']}")

    return " ".join(parts)

# 應用到 DataFrame
df["semantic_text"] = df.apply(build_semantic_text, axis=1)
df

/var/folders/59/tx1m33l955b0h85n6dlwm4440000gn/T/ipykernel_1391/3744166298.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["semantic_text"] = df.apply(build_semantic_text, axis=1)


,author,content,content_length,lang,scrape_time,post_weekday,post_hour,post_period,viral,post_time,...,post_url,emojis,emoji_count,scrape_time_origin,has_question,has_exclaim,has_url,has_mention,has_hashtag,semantic_text
0,ayofvr,Thank you God for another day.,30,en,2025年04月30日 06:27,Wednesday,6,morning,0,2025-04-29T10:58:39.000Z,...,https://www.threads.net/@ayofvr/post/DJBymf8uTrK,,0,2025-04-30 06:27:40.176749+08:00,False,False,False,False,False,author_id: ayofvr emojis: content: Thank you ...
1,NaN,百達翡麗？ 沒有下限的網路病態！,16,Ch,2025年04月30日 06:27,Wednesday,6,morning,1,2025-04-29T12:33:44.000Z,...,https://www.threads.net/@ban.mei.onnnnni/post/...,,0,2025-04-30 06:27:54.964788+08:00,True,True,False,False,False,emojis: content: 百達翡麗？ 沒有下限的網路病態！
2,ribboworld2021,考完期中考，成績都還沒出來，小一女兒就自信的對我說：「我真羨慕妳生了一個天才！」,40,Ch,2025年04月30日 06:28,Wednesday,6,morning,0,2025-04-29T04:39:46.000Z,...,https://www.threads.net/@ribboworld2021/post/D...,,0,2025-04-30 06:28:09.873641+08:00,False,True,False,False,False,author_id: ribboworld2021 topic: 小一日常 emojis: ...
3,ayofvr,Just be strong. Confident. Hopeful. Intellectu...,69,en,2025年04月30日 06:28,Wednesday,6,morning,0,2025-04-29T11:25:22.000Z,...,https://www.threads.net/@ayofvr/post/DJB1qP5OmzP,,0,2025-04-30 06:28:24.726576+08:00,False,False,False,False,False,author_id: ayofvr emojis: content: Just be st...
4,jose_ykc,日文輔系老師上課內容之一AiScReam 歌詞導讀,25,Ch,2025年04月30日 06:28,Wednesday,6,morning,1,2025-04-29T10:36:19.000Z,...,https://www.threads.net/@jose_ykc/post/DJBvpGI...,,0,2025-04-30 06:28:39.393706+08:00,False,False,False,False,False,author_id: jose_ykc topic: 輔仁大學 emojis: conte...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3096,leighton.williams,First tasting in California,27,en,2025年04月30日 00:23,Wednesday,0,night,0,2025年04月29日 07:30,...,https://www.threads.net/@leighton.williams/pos...,,0,2025-04-30 00:23:40.328046+08:00,False,False,False,False,False,author_id: leighton.williams emojis: content:...
3097,cape__man,I hoped it would have been better.,34,en,2025年04月30日 00:23,Wednesday,0,night,0,2025年04月29日 06:31,...,https://www.threads.net/@cape__man/post/DJAdFd...,,0,2025-04-30 00:23:55.063517+08:00,False,False,False,False,False,author_id: cape__man emojis: content: I hoped...
3098,simimoonlight,I can’t wait to watch Beyoncé on TikTok tonigh...,51,en,2025年04月30日 00:24,Wednesday,0,night,1,2025年04月29日 06:26,...,https://www.threads.net/@simimoonlight/post/DJ...,🫶🏿🥹,3,2025-04-30 00:24:39.870994+08:00,False,False,False,False,False,author_id: simimoonlight emojis: 🫶 🏿 🥹 content...
3099,other98,"Tonight, Canada just proved that they have a h...",77,en,2025年04月30日 00:24,Wednesday,0,night,1,2025年04月29日 12:31,...,https://www.threads.net/@other98/post/DJBGV3NxiX_,🙌,1,2025-04-30 00:24:54.669936+08:00,False,True,False,False,False,"author_id: other98 emojis: 🙌 content: Tonight,..."


In [9]:
from transformers import BertTokenizer
import torch
from torch.utils.data import Dataset, DataLoader

# 1. 載入 tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")

# 2. 自訂 Dataset 類別
class TextDataset(Dataset):
    def __init__(self, texts, max_len=128):
        self.encodings = tokenizer(
            texts, 
            truncation=True, 
            padding='max_length', 
            max_length=max_len, 
            return_tensors="pt"
        )

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx]
        }

# 3. 建立 dataset 和 dataloader
texts = df["content"].astype(str).tolist()
text_dataset = TextDataset(texts)
text_loader = DataLoader(text_dataset, batch_size=32)

import torch.nn as nn
import torch.nn.functional as F

class SelfAttentionEncoder(nn.Module):
    def __init__(self, vocab_size, embed_dim=768, num_heads=4):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.attention = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, batch_first=True)
        self.norm = nn.LayerNorm(embed_dim)
        self.fc = nn.Linear(embed_dim, embed_dim)  # 可選加強轉換

    def forward(self, input_ids, attention_mask):
        x = self.embedding(input_ids)  # [B, T, D]
        attn_output, _ = self.attention(x, x, x, key_padding_mask=~attention_mask.bool())
        x = self.norm(attn_output + x)
        x = self.fc(x)
        cls_rep = x[:, 0, :] 
        return cls_rep

# 初始化模型
vocab_size = tokenizer.vocab_size
model = SelfAttentionEncoder(vocab_size)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# 提取語意向量 Z_text
Z_text_list = []

model.eval()
with torch.no_grad():
    for batch in text_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        Z_text = model(input_ids, attention_mask)  # [B, 768]
        Z_text_list.append(Z_text.cpu())

Z_text_tensor = torch.cat(Z_text_list, dim=0)  # [N, 768]


In [13]:
import pandas as pd

Z_text_df = pd.DataFrame(Z_text_tensor.numpy())
Z_text_df

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,1.185373,0.116477,0.122377,0.404507,-0.339129,-0.347968,-0.377152,-0.266511,0.258730,-0.805666,...,0.437772,1.143777,-0.130535,0.552972,0.508016,-0.661004,0.648708,0.508159,0.471049,0.812747
1,1.118061,0.158701,0.106780,0.251318,-0.336016,-0.447374,-0.266132,-0.184926,0.391666,-0.695012,...,0.470352,1.147053,-0.160061,0.309530,0.539303,-0.772830,0.498029,0.462456,0.497207,0.803436
2,1.160336,0.090955,0.141805,0.271038,-0.268028,-0.443410,-0.325646,-0.121320,0.362589,-0.728238,...,0.423751,1.035433,-0.356109,0.497018,0.418673,-0.695262,0.535084,0.468965,0.534107,0.722016
3,1.076843,0.105067,0.052723,0.331179,-0.202970,-0.429289,-0.354241,-0.151234,0.392680,-0.755461,...,0.332896,1.001550,-0.155210,0.619413,0.462634,-0.595787,0.625809,0.477340,0.576133,0.758163
4,1.138116,0.084057,0.129734,0.177018,-0.317661,-0.374959,-0.301814,-0.134798,0.365255,-0.658869,...,0.398198,1.032260,-0.304353,0.398853,0.552792,-0.693314,0.558638,0.415872,0.535430,0.751466
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3096,1.089460,0.230556,0.096753,0.355270,-0.262501,-0.373350,-0.388324,-0.285172,0.496985,-0.821012,...,0.436395,1.199498,-0.107245,0.401726,0.564576,-0.815372,0.613472,0.482972,0.583095,0.766825
3097,1.079693,0.206142,0.059271,0.246766,-0.208253,-0.328518,-0.396139,-0.210764,0.441348,-0.776673,...,0.388379,0.950016,-0.213405,0.464987,0.501455,-0.790398,0.626000,0.466310,0.540599,0.711200
3098,1.129379,0.108640,0.122049,0.300188,-0.298384,-0.332469,-0.461424,-0.200608,0.412536,-0.822279,...,0.437023,1.012208,-0.170115,0.577867,0.450494,-0.554175,0.630714,0.495547,0.548192,0.719363
3099,1.063507,0.102142,0.152797,0.225523,-0.329202,-0.329178,-0.355302,-0.101184,0.401613,-0.827614,...,0.340202,1.069342,-0.156227,0.479461,0.446085,-0.610415,0.554650,0.437024,0.644774,0.715786


## 清洗數據embbedding

In [ ]:
from datasets import Dataset

# =============== BERT 向量嵌入 ===============
df = df.dropna(subset=['content']) #要先處理content空值才能embedding
# --- 載入 tokenizer & model ---
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
model = AutoModel.from_pretrained("bert-base-chinese")
# 選擇硬體設備（MPS、CUDA、CPU），自動判斷是否可用 GPU（M1/M2 晶片上的 MPS 或 CUDA），否則 fallback 到 CPU
device = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# --- 建立 HuggingFace Dataset ---
hf_dataset = Dataset.from_pandas(df[["content"]])

# --- tokenize function ---
def tokenize_function(examples):
    return tokenizer(examples['content'], truncation=True, padding='max_length', max_length=128)
tokenized_dataset = hf_dataset.map(tokenize_function, batched=True)

# --- 取得 [CLS] 向量 ---
def extract_embeddings(batch):
    inputs = {k: torch.tensor(v).to(model.device) for k, v in batch.items() if k in tokenizer.model_input_names}
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state[:, 0, :].detach().cpu().numpy()
    return {"embeddings": embeddings}

# --- 批次轉換為 embeddings ---
batch_size = 64
embeddings_dataset = tokenized_dataset.map(extract_embeddings, batched=True, batch_size=batch_size)

# =============== 匯出最終結果 ===============
# embeddings_dataset["embeddings"] 是 list of 768-dim vectors
embedding_df = pd.DataFrame(embeddings_dataset["embeddings"])
final_df = pd.concat([df.reset_index(drop=True), embedding_df], axis=1)

# 儲存
# final_df.to_csv("C:/Users/User/Desktop/louis/threads_with_embeddings.csv",encoding='utf_8_sig', index=False)
final_df.to_csv("threads_with_embeddings.csv",encoding='utf_8_sig', index=False)
print("✅ 全部處理完成，已輸出 threads_with_embeddings.csv")

Map:   0%|          | 0/3093 [00:00<?, ? examples/s]

Map:   0%|          | 0/3093 [00:00<?, ? examples/s]

✅ 全部處理完成，已輸出 threads_with_embeddings.csv


## 分詞

In [9]:
stopwords = set(['的', '了', '在', '是', '和', '也', '與', '有', '為', '等'])

def tokenize_and_remove_stopwords(text):
    if not isinstance(text, str):
        return ''
    words = jieba.cut(text)
    words_filtered = [word for word in words if word not in stopwords]
    return ' '.join(words_filtered)


df['processed_content'] = df['content'].apply(tokenize_and_remove_stopwords)
df['processed_content'][2]

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/d4/dpw_wc9n69zg41k6pqzpwzcr0000gn/T/jieba.cache
Loading model cost 0.258 seconds.
Prefix dict has been built successfully.


'考完 期中考 ， 成績 都 還沒出 來 ， 小一 女兒 就 自信 對 我 說 ： 「 我 真羨慕 妳 生 一個 天才 ！ 」'

## 機器學習建模

In [10]:
# 計算 TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['processed_content'])

# 計算 TF
tf_vectorizer = CountVectorizer()
tf_matrix = tf_vectorizer.fit_transform(df['processed_content'])

print(tfidf_matrix)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 33647 stored elements and shape (3093, 12887)>
  Coords	Values
  (0, 2810)	0.4662798616255825
  (0, 3199)	0.2941078724734809
  (0, 1316)	0.4428271644129856
  (0, 1220)	0.31033983594608366
  (0, 422)	0.5030766701843938
  (0, 902)	0.3880603079298958
  (1, 9706)	0.44772643118600786
  (1, 10484)	0.44772643118600786
  (1, 8824)	0.25695349239444143
  (1, 3502)	0.425969976146039
  (1, 10379)	0.3887770705954321
  (1, 9624)	0.44772643118600786
  (2, 10526)	0.31144587501432597
  (2, 8426)	0.32887477885054034
  (2, 7301)	0.285391148181221
  (2, 12083)	0.3586696867544276
  (2, 6469)	0.3586696867544276
  (2, 6055)	0.2990798709466531
  (2, 10693)	0.32887477885054034
  (2, 9857)	0.3586696867544276
  (2, 3232)	0.1812649560447855
  (2, 5971)	0.31144587501432597
  (3, 1638)	0.26167811005929115
  (3, 548)	0.23061394459754261
  (3, 2710)	0.3728557459198426
  :	:
  (3090, 3045)	0.4140018744676121
  (3090, 3075)	0.3876142688527415
  (3090, 2883)	

# 多模型分類實驗

In [11]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler, random_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModel
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 8
num_epochs = 5
# ========== 參數設定 ==========
model_tokenizer_map = {
    "FusionMacBERT": "hfl/chinese-macbert-base",
    "PureMacBERT": "hfl/chinese-macbert-base",
    "NumericOnly": None,
    "BiLSTMWithNumeric": "bert-base-chinese",
    "MacBERTWithGRU": "hfl/chinese-macbert-base",
    "MacBERTMLPFusion": "hfl/chinese-macbert-base",
    "TextCNNMacBERT": "hfl/chinese-macbert-base",
    "RoBERTa": "hfl/chinese-roberta-wwm-ext",
    "BERTwwmExt": "hfl/chinese-bert-wwm-ext",
    "ERNIE": "nghuyong/ernie-3.0-base-zh",
    "ConvBERT": "YituTech/conv-bert-base"
}

#tokenizer
default_tokenizer_name = model_tokenizer_map["FusionMacBERT"]
tokenizer = AutoTokenizer.from_pretrained(default_tokenizer_name)

#載入資料
# df = pd.read_csv("C:/Users/User/Desktop/louis/threads_cleaned_v2.csv", encoding='utf_8_sig')
#df = df.dropna(subset=['content', 'view_count']).reset_index(drop=True)
df

,author,content,content_length,lang,scrape_time,post_weekday,post_hour,post_period,viral,post_time,...,post_url,emojis,emoji_count,scrape_time_origin,has_question,has_exclaim,has_url,has_mention,has_hashtag,processed_content
0,ayofvr,Thank you God for another day.,30,en,2025年04月30日 06:27,Wednesday,6,morning,0,2025-04-29T10:58:39.000Z,...,https://www.threads.net/@ayofvr/post/DJBymf8uTrK,,0,2025-04-30 06:27:40.176749+08:00,False,False,False,False,False,Thank you God for another day .
1,NaN,百達翡麗？ 沒有下限的網路病態！,16,Ch,2025年04月30日 06:27,Wednesday,6,morning,1,2025-04-29T12:33:44.000Z,...,https://www.threads.net/@ban.mei.onnnnni/post/...,,0,2025-04-30 06:27:54.964788+08:00,True,True,False,False,False,百達 翡麗 ？ 沒有 下限 網路 病態 ！
2,ribboworld2021,考完期中考，成績都還沒出來，小一女兒就自信的對我說：「我真羨慕妳生了一個天才！」,40,Ch,2025年04月30日 06:28,Wednesday,6,morning,0,2025-04-29T04:39:46.000Z,...,https://www.threads.net/@ribboworld2021/post/D...,,0,2025-04-30 06:28:09.873641+08:00,False,True,False,False,False,考完 期中考 ， 成績 都 還沒出 來 ， 小一 女兒 就 自信 對 我 說 ： 「 我 真...
3,ayofvr,Just be strong. Confident. Hopeful. Intellectu...,69,en,2025年04月30日 06:28,Wednesday,6,morning,0,2025-04-29T11:25:22.000Z,...,https://www.threads.net/@ayofvr/post/DJB1qP5OmzP,,0,2025-04-30 06:28:24.726576+08:00,False,False,False,False,False,Just be strong . Confident . Hopeful ....
4,jose_ykc,日文輔系老師上課內容之一AiScReam 歌詞導讀,25,Ch,2025年04月30日 06:28,Wednesday,6,morning,1,2025-04-29T10:36:19.000Z,...,https://www.threads.net/@jose_ykc/post/DJBvpGI...,,0,2025-04-30 06:28:39.393706+08:00,False,False,False,False,False,日文 輔系 老師 上 課內容 之一 AiScReam 歌詞 導讀
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3096,leighton.williams,First tasting in California,27,en,2025年04月30日 00:23,Wednesday,0,night,0,2025年04月29日 07:30,...,https://www.threads.net/@leighton.williams/pos...,,0,2025-04-30 00:23:40.328046+08:00,False,False,False,False,False,First tasting in California
3097,cape__man,I hoped it would have been better.,34,en,2025年04月30日 00:23,Wednesday,0,night,0,2025年04月29日 06:31,...,https://www.threads.net/@cape__man/post/DJAdFd...,,0,2025-04-30 00:23:55.063517+08:00,False,False,False,False,False,I hoped it would have been better .
3098,simimoonlight,I can’t wait to watch Beyoncé on TikTok tonigh...,51,en,2025年04月30日 00:24,Wednesday,0,night,1,2025年04月29日 06:26,...,https://www.threads.net/@simimoonlight/post/DJ...,🫶🏿🥹,3,2025-04-30 00:24:39.870994+08:00,False,False,False,False,False,I can ’ t wait to watch Beyonc é o...
3099,other98,"Tonight, Canada just proved that they have a h...",77,en,2025年04月30日 00:24,Wednesday,0,night,1,2025年04月29日 12:31,...,https://www.threads.net/@other98/post/DJBGV3NxiX_,🙌,1,2025-04-30 00:24:54.669936+08:00,False,True,False,False,False,"Tonight , Canada just proved that th..."


# Label 分群 (用四分位數分三群)

In [ ]:
# Label 分群 ：標籤轉換（按瀏覽數進行分群）
# 取「瀏覽數」的第 80 百分位作為高人氣門檻（q_high）、第 20 百分位作為低人氣門檻（q_low）
# 把每筆資料的「view_count」劃分為三類：0 高人氣 (high)、1 中人氣 (medium)、2 低人氣 (low)
q_high = df['view_count'].quantile(0.80)
q_low = df['view_count'].quantile(0.20)
df['view_class'] = df['view_count'].apply(lambda x: "high" if x >= q_high else ("low" if x <= q_low else "medium"))
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['view_class'])

#做 oversampling 類別資料平衡 ：資料增強（針對 high / low 類別 oversample）
df_high = df[df['view_class'] == 'high']
df_low = df[df['view_class'] == 'low']
df_medium = df[df['view_class'] == 'medium']

# 分別取出三個分類的樣本：對 high 與 low 分類做「過採樣」，各自複製三次，讓資料數量接近 medium
# 再對整個資料表做隨機打散 (shuffle），避免模型學到資料順序的偏誤
df_high_oversampled = pd.concat([df_high] * 3, ignore_index=True)
df_low_oversampled = pd.concat([df_low] * 3, ignore_index=True)
df = pd.concat([df_medium, df_high_oversampled, df_low_oversampled], ignore_index=True)
df = df.sample(frac=1).reset_index(drop=True)


In [ ]:
q_high

In [ ]:
q_low

## Label 分群 （1000以下、1000~10000、10000~100000、100000以上)

In [12]:
# 將 view_count 分成四類：
# 0: 小於 1000
# 1: 1000 ~ 9999
# 2: 10000 ~ 99999
# 3: 100000 以上

def map_view_class(x):
    if x < 1000:
        return 'low'
    elif x < 10000:
        return 'medium'
    elif x < 100000:
        return 'high'
    else:
        return 'very_high'

df['view_class'] = df['view_count'].apply(map_view_class)

# 編碼成數字 label
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['view_class'])

df_high = df[df['view_class'] == 'very_high']
df_medium = df[df['view_class'] == 'high']
df_low = df[df['view_class'] == 'medium']
df_very_low = df[df['view_class'] == 'low']

# 針對較少的類別進行擴增（假設 high 和 very_low 比較少）
df_high_oversampled = pd.concat([df_high] * 3, ignore_index=True)
df_very_low_oversampled = pd.concat([df_very_low] * 3, ignore_index=True)

# 合併並打亂
df = pd.concat([df_medium, df_low, df_high_oversampled, df_very_low_oversampled], ignore_index=True)
df = df.sample(frac=1).reset_index(drop=True)

df

,author,content,content_length,lang,scrape_time,post_weekday,post_hour,post_period,viral,post_time,...,emoji_count,scrape_time_origin,has_question,has_exclaim,has_url,has_mention,has_hashtag,processed_content,view_class,label
0,031758_wu,老實說我現在還是搞不懂一場演唱會搶這麼多次票的意義在哪裡💧,29,Ch,2025年04月30日 08:30,Wednesday,8,morning,0,2025-04-29T12:59:30.000Z,...,1,2025-04-30 08:30:58.388029+08:00,False,False,False,False,False,老實 說 我現 還是 搞不懂 一場 演唱 會 搶 這麼 多次 票 意義在 哪裡 💧,low,1
1,yuii_1.6,吳海嫄🫧說能看到很多恩瑟很開心還說台北的應援法太讚了 ㅠㅠ,29,Ch,2025年04月29日 02:17,Tuesday,2,night,0,2025年04月28日 02:23,...,1,2025-04-29 02:17:59.586251+08:00,False,False,False,False,False,吳海 嫄 🫧 說 能 看到 很多 恩瑟 很 開心 還說 台北 應援法 太 讚 ㅠ ㅠ,medium,2
2,rejiya1575,Tahajjud is life changer. ♥,27,lg,2025年04月28日 05:19,Monday,5,morning,0,2025年04月27日 18:08,...,1,2025-04-28 05:19:53.327702+08:00,False,False,False,False,False,Tahajjud is life changer . ♥,low,1
3,jonblta,i dont wanna hear Warriors fans complaining ab...,87,en,2025年05月03日 19:29,Saturday,19,evening,0,2025-05-03T03:09:05.000Z,...,0,2025-05-03 19:29:20.954467+08:00,False,False,False,False,False,i dont wanna hear Warriors fans co...,low,1
4,_lync.m,Trải nghiệm đầu tiên khi đu Em Xinh: Yeolan và...,80,vi,2025年05月03日 08:40,Saturday,8,morning,1,2025-05-02T11:43:24.000Z,...,0,2025-05-03 08:40:12.796089+08:00,False,False,False,False,False,Tr ả i nghi ệ m đ ầ u ti ê n khi đ u...,high,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6222,jauyiwu,連續三天走在路上，聽到不同的人說要去白沙屯。￼我想，這已經變成台灣人一個新的時間體感，就好像...,57,Ch,2025年05月01日 17:40,Thursday,17,evening,0,2025-05-01T00:46:32.000Z,...,0,2025-05-01 17:40:50.710474+08:00,False,False,False,False,False,連續 三天 走 路上 ， 聽到 不同 人 說 要 去 白沙 屯 。 ￼ 我 想 ， 這已 經...,medium,2
6223,__mi_achool,收到vip 了 原價讓一張 A2 2XX,20,Ch,2025年04月29日 07:25,Tuesday,7,morning,0,2025年04月28日 17:58,...,0,2025-04-29 07:25:02.893557+08:00,False,False,False,False,False,收到 vip 原價 讓 一張 A2 2XX,low,1
6224,lhs_7632,分手第十八天 偶然看見你的限時動態，這是十八天內換的第二個男生了，打開了你追蹤名單 不管是我...,124,Ch,2025年04月30日 07:03,Wednesday,7,morning,0,2025-04-28T22:51:13.000Z,...,0,2025-04-30 07:03:46.817921+08:00,False,False,False,False,False,分手 第十八天 偶然 看見 你 限時 動態 ， 這是 十八天 內換 第二 個 男生 ， ...,medium,2
6225,taro_puree_05,我有說我要去嗎 你就給我第一排的票 虫合❓,21,Ch,2025年05月01日 03:09,Thursday,3,night,0,2025-04-30T05:43:31.000Z,...,1,2025-05-01 03:09:51.931280+08:00,False,False,False,False,False,我 說 我 要 去 嗎 你 就給 我 第一排 票 虫合 ❓,medium,2


In [ ]:
class RegressionDataset(Dataset):
    def __init__(self, df):
        self.input_ids = df['input_ids'].tolist()
        self.attention_mask = df['attention_mask'].tolist()
        self.numerics = df[num_cols].values
        self.targets = df['target'].values

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_mask[idx], dtype=torch.long),
            'numerics': torch.tensor(self.numerics[idx], dtype=torch.float),
            'target': torch.tensor(self.targets[idx], dtype=torch.float)
        }


# Normalization 數值特徵標準化

In [13]:
# Normalization 數值特徵標準化
base_num_cols = ['like_count', 'share_count', 'repost_count', 'reply_count', 'emoji_count', 'has_photo', 'has_video', 'has_question', 'has_exclaim', 'has_mention', 'has_url', 'has_hashtag', 'content_length']
# 找出 one-hot 編碼的欄位（語言類型、發文時段、星期幾等類別欄位）
# 使用 StandardScaler 將數值欄位轉換為「標準常態分布」（mean=0, std=1），有助於模型學習穩定。
onehot_cols = [col for col in df.columns if col.startswith('lang_') or col.startswith('post_period_') or col.startswith('post_weekday_')]
num_cols = base_num_cols + onehot_cols
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

# Tokenizer 文本編碼 ：使用事先定義好的 tokenizer（例如 MacBERT、RoBERTa）對貼文進行斷詞、編碼
# 將編碼後的結果儲存到 df 中，這兩個欄位會作為 BERT 模型的輸入
encodings = tokenizer(df['content'].tolist(), truncation=True, padding='max_length', max_length=128)
df['input_ids'] = encodings['input_ids']  # 斷詞後對應的詞彙 ID
df['attention_mask'] = encodings['attention_mask']  # 對應位置是否是 padding（0）或實際內容（1）

In [ ]:
df

In [ ]:
# Dataset
class CustomDataset(Dataset):
    def __init__(self, df):
        self.input_ids = df['input_ids'].tolist()
        self.attention_mask = df['attention_mask'].tolist()
        self.labels = df['label'].tolist()
        self.numerics = df[num_cols].values

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_mask[idx], dtype=torch.long),
            'numerics': torch.tensor(self.numerics[idx], dtype=torch.float),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# Loss
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0):
        super().__init__()
        self.gamma = gamma
        self.ce = nn.CrossEntropyLoss()

    def forward(self, input, target):
        logp = self.ce(input, target)
        p = torch.exp(-logp)
        loss = (1 - p) ** self.gamma * logp
        return loss.mean()

#模型架構

In [ ]:
#模型架構
# 1. FusionMacBERT：BERT + 數值特徵 concat
class FusionMacBERTModel(nn.Module):
    def __init__(self, model_name, num_numeric_features, num_classes):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        self.numeric_fc = nn.Linear(num_numeric_features, 64)
        self.classifier = nn.Linear(self.bert.config.hidden_size + 64, num_classes)

    def forward(self, input_ids, attention_mask, numerics):
        cls_output = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :]
        num_out = torch.relu(self.numeric_fc(numerics))
        combined = torch.cat((cls_output, num_out), dim=1)
        return self.classifier(self.dropout(combined))

# 2. PureMacBERT：只有文字
class PureMacBERTModel(nn.Module):
    def __init__(self, model_name, num_classes):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask, numerics=None):
        cls_output = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :]
        return self.classifier(self.dropout(cls_output))

# 3. NumericOnly：只有數值特徵
class NumericOnlyModel(nn.Module):
    def __init__(self, num_numeric_features, num_classes):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(num_numeric_features, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, num_classes)
        )

    def forward(self, input_ids=None, attention_mask=None, numerics=None):
        return self.classifier(numerics)

# 4. BiLSTMWithNumeric：LSTM 處理詞嵌入 + 數值特徵
class BiLSTMWithNumeric(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_numeric_features, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.numeric_fc = nn.Linear(num_numeric_features, 64)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(hidden_dim * 2 + 64, num_classes)

    def forward(self, input_ids, attention_mask, numerics):
        x = self.embedding(input_ids)
        lstm_out, _ = self.lstm(x)
        pooled = lstm_out[:, -1, :]
        num_out = torch.relu(self.numeric_fc(numerics))
        combined = torch.cat((pooled, num_out), dim=1)
        return self.classifier(self.dropout(combined))

# 5. MacBERTWithGRU：BERT + GRU + 數值特徵
class MacBERTWithGRU(nn.Module):
    def __init__(self, model_name, num_numeric_features, num_classes):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.gru = nn.GRU(self.bert.config.hidden_size, 128, batch_first=True, bidirectional=True)
        self.numeric_fc = nn.Linear(num_numeric_features, 64)
        self.classifier = nn.Linear(128*2 + 64, num_classes)
        self.dropout = nn.Dropout(0.3)

    def forward(self, input_ids, attention_mask, numerics):
        bert_out = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state
        gru_out, _ = self.gru(bert_out)
        pooled = gru_out[:, -1, :]
        num_out = torch.relu(self.numeric_fc(numerics))
        combined = torch.cat((pooled, num_out), dim=1)
        return self.classifier(self.dropout(combined))

# 6. MacBERTMLPFusion：BERT + 數值特徵 -> MLP
class MacBERTMLPFusion(nn.Module):
    def __init__(self, model_name, num_numeric_features, num_classes):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.fc = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size + num_numeric_features, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, input_ids, attention_mask, numerics):
        cls_output = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :]
        combined = torch.cat((cls_output, numerics), dim=1)
        return self.fc(combined)

# 7. TextCNNMacBERT：BERT 輸出卷積後 + 數值特徵
class TextCNNMacBERT(nn.Module):
    def __init__(self, model_name, num_numeric_features, num_classes):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.convs = nn.ModuleList([nn.Conv2d(1, 64, (k, self.bert.config.hidden_size)) for k in [2, 3, 4]])
        self.numeric_fc = nn.Linear(num_numeric_features, 64)
        self.classifier = nn.Linear(64 * len([2, 3, 4]) + 64, num_classes)

    def conv_and_pool(self, x, conv):
        x = torch.relu(conv(x)).squeeze(3)
        x = torch.max_pool1d(x, x.size(2)).squeeze(2)
        return x

    def forward(self, input_ids, attention_mask, numerics):
        x = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state.unsqueeze(1)
        cnn_out = torch.cat([self.conv_and_pool(x, conv) for conv in self.convs], 1)
        num_out = torch.relu(self.numeric_fc(numerics))
        combined = torch.cat((cnn_out, num_out), dim=1)
        return self.classifier(combined)


In [ ]:
#訓練與評估
def train_and_eval(model, name, preview_count=10):
    model = model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

    loss_fn = FocalLoss()
    # loss_fn = nn.MSELoss()

    # 訓練階段
    for epoch in range(5):
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            numerics = batch['numerics'].to(device)
            labels = batch['labels'].to(device)
            output = model(input_ids, attention_mask, numerics)
            loss = loss_fn(output, labels)
            loss.backward()
            optimizer.step()

    # 評估階段
    model.eval()
    all_preds, all_labels = [], []
    '''
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            numerics = batch['numerics'].to(device)
            labels = batch['labels'].to(device)
            output = model(input_ids, attention_mask, numerics)
            preds = output.argmax(dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    from sklearn.metrics import mean_squared_error, mean_absolute_error
    mse = mean_squared_error(all_targets, all_preds)
    mae = mean_absolute_error(all_targets, all_preds)
    print(f"[{name} 評估結果] MSE: {mse:.2f} | MAE: {mae:.2f}")
    print(f"\n{name} 評估結果：")
    print(classification_report(all_labels, all_preds, target_names=label_encoder.classes_))
    '''
    preview_shown = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            numerics = batch['numerics'].to(device)
            labels = batch['labels'].to(device)
            output = model(input_ids, attention_mask, numerics)
            preds = output.argmax(dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

            #印出前幾筆的預測、真實值
            if preview_shown < preview_count:
                batch_size = input_ids.shape[0]
                for i in range(batch_size):
                    if preview_shown >= preview_count:
                        break
                    input_id = input_ids[i].cpu().numpy()
                    text = tokenizer.decode(input_id, skip_special_tokens=True)
                    print(f"\n[{name} 預測] 第 {preview_shown+1} 筆")
                    print(f"Text: {text}")
                    print(f"Predicted: {label_encoder.inverse_transform([preds[i]])[0]}")
                    print(f"Actual:    {label_encoder.inverse_transform([labels[i].cpu().item()])[0]}")
                    preview_shown += 1

    print(f"\n{name} 評估結果：")
    print(classification_report(all_labels, all_preds, target_names=label_encoder.classes_))

In [ ]:
# 資料分割：資料集切分與取樣
dataset = CustomDataset(df)
train_size = int(0.8 * len(dataset))
train_dataset, test_dataset = random_split(dataset, [train_size, len(dataset)-train_size])
train_labels = [train_dataset[i]['labels'].item() for i in range(len(train_dataset))]
class_counts = pd.Series(train_labels).value_counts().to_dict()
weights = [1.0 / class_counts[label] for label in train_labels]
sampler = WeightedRandomSampler(weights, len(weights), replacement=True)
train_loader = DataLoader(train_dataset, batch_size=8, sampler=sampler)
test_loader = DataLoader(test_dataset, batch_size=8)

# 執行多模型訓練
model_variants = {
    "FusionMacRegressor": FusionMacBERTRegressor("hfl/chinese-macbert-base", len(num_cols), 3),
    "FusionMacBERT": FusionMacBERTModel("hfl/chinese-macbert-base", len(num_cols), 3),
    "PureMacBERT": PureMacBERTModel("hfl/chinese-macbert-base", 3),
    "NumericOnly": NumericOnlyModel(len(num_cols), 3),
    "BiLSTMWithNumeric": BiLSTMWithNumeric(tokenizer.vocab_size, 128, 128, len(num_cols), 3),
    "MacBERTWithGRU": MacBERTWithGRU("hfl/chinese-macbert-base", len(num_cols), 3),
    "MacBERTMLPFusion": MacBERTMLPFusion("hfl/chinese-macbert-base", len(num_cols), 3),
    "TextCNNMacBERT": TextCNNMacBERT("hfl/chinese-macbert-base", len(num_cols), 3),
    "RoBERTa": FusionMacBERTModel("hfl/chinese-roberta-wwm-ext", len(num_cols), 3),
    "BERTwwmExt": FusionMacBERTModel("hfl/chinese-bert-wwm-ext", len(num_cols), 3),
    "ERNIE": FusionMacBERTModel("nghuyong/ernie-3.0-base-zh", len(num_cols), 3),
    "ConvBERT": FusionMacBERTModel("YituTech/conv-bert-base", len(num_cols), 3)
}

# 逐個模型訓練與輸出結果
for name, model in model_variants.items():
    tokenizer_name = model_tokenizer_map.get(name, default_tokenizer_name)
    if tokenizer_name:
        tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        encodings = tokenizer(df['content'].tolist(), truncation=True, padding='max_length', max_length=128)
        df['input_ids'] = encodings['input_ids']
        df['attention_mask'] = encodings['attention_mask']
    train_and_eval(model, name)

1. FusionMacBERT ✅文字 + ✅數值
BERT： 使用 MacBERT
架構： 把 [CLS] 向量與數值特徵經過 MLP 融合
用途： 做為 baseline 融合模型
優點： 同時考慮內容語義與貼文統計資料（如按讚數、是否有 hashtag）

2. PureMacBERT ✅文字 + ❌數值
BERT： 使用 MacBERT
架構： 單純使用 [CLS]，後接 linear 層分類
用途： 純語言模型 baseline
對照： 可用來比較是否有數值輔助提升效果

3. NumericOnly ❌文字 + ✅數值
模型類型： 只有數值輸入，經過 MLP 做分類
用途： 測試「只靠貼文統計資料」能否達到合理分類
對照： 可與文字模型或融合模型對比效果

4. BiLSTMWithNumeric ✅文字（Embedding+LSTM）+ ✅數值
嵌入方式： 使用 nn.Embedding + BiLSTM 處理文字（不是 BERT）
融合方式： 將 LSTM 最後時間步 + 數值特徵拼接
特別點： 測試「非 Transformer 模型」是否仍具競爭力

5. MacBERTWithGRU ✅文字（MacBERT）+ ✅數值
文字處理： MacBERT 之後再串 GRU
融合方式： GRU 輸出最後一步拼接數值特徵
意圖： 想看 BERT+RNN 的表現 vs. 傳統 BERT

6. MacBERTMLPFusion ✅文字 + ✅數值
處理方式： 文字與數值直接拼接後進入 MLP
不同於 FusionMacBERT：
沒有額外處理數值特徵（如沒有經過 nn.Linear)
更單純的融合設計（屬於 Early Fusion）

7. TextCNNMacBERT ✅文字 + ✅數值
模型組合：
使用 BERT 編碼後丟進 CNN filter (TextCNN)
再與數值特徵融合
用途： 測試 BERT 結合 CNN 特徵提取是否提升效果
有趣點： 有些短文模型（如微博、Threads）對 CNN 特徵抓取敏感

8. RoBERTa ✅文字 + ✅數值
BERT 替代品： 改用 RoBERTa（中文版本）
融合方式： 同 FusionMacBERT
實驗目的： 測試不同語言模型對結果的影響（語言模型 ablation）


9. BERTwwmExt ✅文字 + ✅數值
BERT： 使用 Chinese BERT whole-word-masking 擴展版
比較目的： 同上，用於測試不同語言模型特性的影響

10. ERNIE ✅文字 + ✅數值
BERT： 改用百度的 ERNIE（引入知識增強）
適用場景： 當文本與常識有關（如話題、用語）
目的： 評估知識型語言模型在社群文本分類的效果

11. ConvBERT ✅文字 + ✅數值
模型特色： 使用 Convolution + Self-Attention 混合架構的 BERT
實驗意義： 試驗非傳統 Self-Attention 模型是否有優勢


| 模型名稱              | 說明               | 是否融合 | 文本處理法         | 特殊處理       |
| ----------------- | ---------------- | ---- | ------------- | ---------- |
| FusionMacBERT     | BERT + 數值特徵      | ✅    | MacBERT       | 自製融合層      |
| PureMacBERT       | 純文本模型            | ❌    | MacBERT       | baseline   |
| NumericOnly       | 純統計數值            | ❌    | 無             | MLP only   |
| BiLSTMWithNumeric | LSTM + 數值        | ✅    | nn.Embedding  | 不使用 BERT   |
| MacBERTWithGRU    | BERT + GRU + 數值  | ✅    | MacBERT + GRU | 時序特徵強化     |
| MacBERTMLPFusion  | BERT + 數值        | ✅    | MacBERT       | 拼接後進 MLP   |
| TextCNNMacBERT    | BERT + CNN + 數值  | ✅    | MacBERT + CNN | 模仿 TextCNN |
| RoBERTa           | 換 BERT backbone  | ✅    | RoBERTa       | 模型比較       |
| BERTwwmExt        | 換 BERT backbone  | ✅    | BERT-wwm      | 模型比較       |
| ERNIE             | 引入知識的 BERT       | ✅    | ERNIE         | 模型比較       |
| ConvBERT          | 混合卷積 + 注意力的 BERT | ✅    | ConvBERT      | 模型比較       |




In [ ]:
# 資料分割：資料集切分與取樣
dataset = CustomDataset(df)
train_size = int(0.8 * len(dataset))
train_dataset, test_dataset = random_split(dataset, [train_size, len(dataset)-train_size])
train_labels = [train_dataset[i]['labels'].item() for i in range(len(train_dataset))]
class_counts = pd.Series(train_labels).value_counts().to_dict()
weights = [1.0 / class_counts[label] for label in train_labels]
sampler = WeightedRandomSampler(weights, len(weights), replacement=True)
train_loader = DataLoader(train_dataset, batch_size=8, sampler=sampler)
test_loader = DataLoader(test_dataset, batch_size=8)

# 執行多模型訓練
model_variants = {
    "FusionMacBERT": FusionMacBERTModel("hfl/chinese-macbert-base", len(num_cols), 3),
    "PureMacBERT": PureMacBERTModel("hfl/chinese-macbert-base", 3),
    "NumericOnly": NumericOnlyModel(len(num_cols), 3),
    "BiLSTMWithNumeric": BiLSTMWithNumeric(tokenizer.vocab_size, 128, 128, len(num_cols), 3),
    "MacBERTWithGRU": MacBERTWithGRU("hfl/chinese-macbert-base", len(num_cols), 3),
    "MacBERTMLPFusion": MacBERTMLPFusion("hfl/chinese-macbert-base", len(num_cols), 3),
    "TextCNNMacBERT": TextCNNMacBERT("hfl/chinese-macbert-base", len(num_cols), 3),
    "RoBERTa": FusionMacBERTModel("hfl/chinese-roberta-wwm-ext", len(num_cols), 3),
    "BERTwwmExt": FusionMacBERTModel("hfl/chinese-bert-wwm-ext", len(num_cols), 3),
    "ERNIE": FusionMacBERTModel("nghuyong/ernie-3.0-base-zh", len(num_cols), 3),
    "ConvBERT": FusionMacBERTModel("YituTech/conv-bert-base", len(num_cols), 3)
}

# 逐個模型訓練與輸出結果
for name, model in model_variants.items():
    tokenizer_name = model_tokenizer_map.get(name, default_tokenizer_name)
    if tokenizer_name:
        tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        encodings = tokenizer(df['content'].tolist(), truncation=True, padding='max_length', max_length=128)
        df['input_ids'] = encodings['input_ids']
        df['attention_mask'] = encodings['attention_mask']
    train_and_eval(model, name)

# 迴歸預測

In [14]:
# 原本這樣分類（要拿掉）
# df['view_class'] = ...
# df['label'] = ...

# 直接用原始 view_count 作為 regression target
df = df.dropna(subset=["content", "view_count"])
df['target'] = df['view_count'].apply(parse_count)  # 如果 view_count 不是數字要先轉換


In [15]:
def train_and_eval_regression(model, name):
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
    loss_fn = nn.MSELoss()

    # 訓練
    for epoch in range(5):
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            numerics = batch['numerics'].to(device)
            targets = batch['labels'].float().to(device)  # 重要：labels 必須是 float
            preds = model(input_ids, attention_mask, numerics)
            loss = loss_fn(preds, targets)
            loss.backward()
            optimizer.step()

    # 評估
    model.eval()
    all_preds, all_targets = [], []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            numerics = batch['numerics'].to(device)
            targets = batch['labels'].float().to(device)
            preds = model(input_ids, attention_mask, numerics)
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    from sklearn.metrics import mean_squared_error, mean_absolute_error
    mse = mean_squared_error(all_targets, all_preds)
    mae = mean_absolute_error(all_targets, all_preds)
    r2 = r2_score(all_targets, all_preds)
    print(f"\n{name}  MSE: {mse:.2f} | MAE: {mae:.2f}| R2: {r2:.2f}")

    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

    return all_targets, all_preds





In [16]:
class FusionMacBERTRegressor(nn.Module):
    def __init__(self, model_name, num_numeric_features, output_dim=1):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        self.numeric_fc = nn.Linear(num_numeric_features, 64)
        self.regressor = nn.Linear(self.bert.config.hidden_size + 64, output_dim)

    def forward(self, input_ids, attention_mask, numerics):
        cls_output = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :]
        num_out = torch.relu(self.numeric_fc(numerics))
        combined = torch.cat((cls_output, num_out), dim=1)
        return self.regressor(self.dropout(combined)).squeeze(1)  # (batch,)


In [17]:
rm -rf ~/.cache/huggingface/transformers/hfl__chinese-macbert-base


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [18]:
# Dataset
'''
class CustomDatasetRegression(Dataset):
    def __init__(self, df):
        self.input_ids = df['input_ids'].tolist()
        self.attention_mask = df['attention_mask'].tolist()
        self.labels = df['label'].astype(float).values   # ← 為回歸任務需轉成 float
        self.numerics = df[num_cols].astype(float).values  # ← 確保為 float array

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_mask[idx], dtype=torch.long),
            'numerics': torch.tensor(self.numerics[idx], dtype=torch.float),  # ← 修正為 float
            'labels': torch.tensor(self.labels[idx], dtype=torch.float)       # ← 修正為 float
        }
'''
class CustomDatasetRegression(Dataset):
    def __init__(self, df):
        self.input_ids = df['input_ids'].tolist()
        self.attention_mask = df['attention_mask'].tolist()
        self.labels = df['target'].astype(float).values
        self.numerics = df[num_cols].astype(float).values

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_mask[idx], dtype=torch.long),
            'numerics': torch.tensor(self.numerics[idx], dtype=torch.float),
            'labels': torch.tensor(self.labels[idx], dtype=torch.float)
        }

# Loss
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0):
        super().__init__()
        self.gamma = gamma
        self.ce = nn.CrossEntropyLoss()

    def forward(self, input, target):
        logp = self.ce(input, target)
        p = torch.exp(-logp)
        loss = (1 - p) ** self.gamma * logp
        return loss.mean()

In [19]:

dataset = CustomDatasetRegression(df)
train_size = int(0.8 * len(dataset))
train_dataset, test_dataset = random_split(dataset, [train_size, len(dataset) - train_size])
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)


class FusionMacBERTRegressor(nn.Module):
    def __init__(self, model_name, num_numeric_features, output_dim=1):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        self.numeric_fc = nn.Linear(num_numeric_features, 64)
        self.regressor = nn.Linear(self.bert.config.hidden_size + 64, output_dim)

    def forward(self, input_ids, attention_mask, numerics):
        cls_output = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :]
        num_out = torch.relu(self.numeric_fc(numerics))
        combined = torch.cat((cls_output, num_out), dim=1)
        return self.regressor(self.dropout(combined)).squeeze(1)


def train_and_eval_regression(model, name):
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
    loss_fn = nn.MSELoss()

    for epoch in range(5):
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            numerics = batch['numerics'].to(device)
            targets = batch['labels'].float().to(device)

            preds = model(input_ids, attention_mask, numerics)
            loss = loss_fn(preds, targets)
            loss.backward()
            optimizer.step()

    model.eval()
    all_preds, all_targets = [], []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            numerics = batch['numerics'].to(device)
            targets = batch['labels'].float().to(device)

            preds = model(input_ids, attention_mask, numerics)
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    mse = mean_squared_error(all_targets, all_preds)
    mae = mean_absolute_error(all_targets, all_preds)
    r2 = r2_score(all_targets, all_preds)

    print(f"\n{name}  MSE: {mse:.2f} | MAE: {mae:.2f} | R²: {r2:.2f}")
    return all_targets, all_preds


model = FusionMacBERTRegressor("hfl/chinese-macbert-base", len(num_cols))
all_targets, all_preds = train_and_eval_regression(model, "FusionMacBERTRegressor")


FusionMacBERTRegressor  MSE: 4410895810.43 | MAE: 10676.62 | R²: -0.03


#**下面都是舊的東西而已~~~~**


In [ ]:
stopwords = set(['的', '了', '在', '是', '和', '也', '與', '有', '為', '等'])

def tokenize_and_remove_stopwords(text):
    words = jieba.cut(text)
    words_filtered = [word for word in words if word not in stopwords]
    return ' '.join(words_filtered)

df['processed_content'] = df['Content'].apply(tokenize_and_remove_stopwords)

new_article = "IC 設計大廠聯發科 (2454-TW) 副董事長暨執行長蔡力行今 (26) 日獲頒潘文淵獎，會後受訪表示，聯發科 3 奈米會在台積電 (2330-TW)(TSM-US) 做，且由於先進製程技術相當複雜，不論要採用或更換都非常困難，雙方會持續緊密合作。外界今日提問不論是輝達 (NVDA-US)、蘋果 (AAPL-US) 等都表示尋求多元的晶圓代工方案，蔡力行回應，聯發科在先進製程持續與台積電緊密合作，英特爾 (INTC-US) 則負責 16 奈米蔡力行也強調，聯發科不會只停在採用 4 奈米，也會採用 3 奈米製程，此外，由於電晶體微縮速度趨緩，儘管技術上可行，但不一定符合經濟效益，因此技術也逐步從平面變成 2D、2.5D，甚至 3D 等，先進封裝的重要性比以前增加。至於跟輝達合作，蔡力行重申，雙方合作仍以汽車為主，輝達布局車用比聯發科早，主要著墨在智慧座艙與 ADAS 系統，雙方有很好的配合，其中，輝達主攻高階、聯發科則瞄準中階，雙方正密切合作開會。"
processed_new_article = tokenize_and_remove_stopwords(new_article)

# Random Forest 模型訓練與預測
from sklearn.ensemble import RandomForestClassifier

# 使用相同的數據分割方式
X_train_rf, X_val_rf, y_train_rf, y_val_rf = train_test_split(tfidf_matrix, y, test_size=0.1, random_state=42)

X_train_rf, X_val_rf, y_train_rf, y_val_rf = train_test_split(X_train_rf, y_train_rf, test_size=0.1, random_state=42)

# 創建隨機森林模型
rand_forest_model = RandomForestClassifier(n_estimators=100, random_state=42)

# 訓練隨機森林模型
rand_forest_model.fit(X_train_rf, y_train_rf)

# 預測
y_val_pred_rf = rand_forest_model.predict(X_val_rf)
y_test_pred_rf = rand_forest_model.predict(X_test)

# 分類報告
print("驗證集 Validation Classification Report:")
print(classification_report(y_val_rf, y_val_pred_rf))

print("\n測試集 Test Classification Report:")
print(classification_report(y_test, y_test_pred_rf))

# 對照表
result_df_val_rf = pd.DataFrame({'Actual': y_val_rf, 'Predicted': y_val_pred_rf})
result_df_test_rf = pd.DataFrame({'Actual': y_test, 'Predicted': y_test_pred_rf})

print("驗證集 Validation Result Comparison:")
print(result_df_val_rf)

print("\n測試集 Test Result Comparison:")
print(result_df_test_rf)



In [ ]:
new_article = "IC 設計大廠聯發科 (2454-TW) 副董事長暨執行長蔡力行今 (26) 日獲頒潘文淵獎，會後受訪表示，聯發科 3 奈米會在台積電 (2330-TW)(TSM-US) 做，且由於先進製程技術相當複雜，不論要採用或更換都非常困難，雙方會持續緊密合作。外界今日提問不論是輝達 (NVDA-US)、蘋果 (AAPL-US) 等都表示尋求多元的晶圓代工方案，蔡力行回應，聯發科在先進製程持續與台積電緊密合作，英特爾 (INTC-US) 則負責 16 奈米蔡力行也強調，聯發科不會只停在採用 4 奈米，也會採用 3 奈米製程，此外，由於電晶體微縮速度趨緩，儘管技術上可行，但不一定符合經濟效益，因此技術也逐步從平面變成 2D、2.5D，甚至 3D 等，先進封裝的重要性比以前增加。至於跟輝達合作，蔡力行重申，雙方合作仍以汽車為主，輝達布局車用比聯發科早，主要著墨在智慧座艙與 ADAS 系統，雙方有很好的配合，其中，輝達主攻高階、聯發科則瞄準中階，雙方正密切合作開會。"

processed_new_article = tokenize_and_remove_stopwords(new_article)

print(processed_new_article)

# 將新文章轉換為 TF-IDF 表示形式
new_article_tfidf = tfidf_vectorizer.transform([processed_new_article])

# 使用投票分類器進行預測
predicted_label_ensemble = voting_classifier.predict(new_article_tfidf)

print(f"新文章預測結果: {predicted_label_ensemble}")


In [ ]:
# 使用 inverse_transform 將預測的數字編碼轉換回原始標籤
predicted_label_original = label_encoder.inverse_transform(predicted_label_ensemble)

print(f"新文章預測結果（原始標籤）: {predicted_label_original}")


## 實際預測（研究）


In [ ]:
# 獲取所有標籤對應的編碼
all_labels = label_encoder.classes_

print("所有標籤對應的編碼:")
for label_code, label in enumerate(all_labels):
    print(f"編碼 {label_code}: 標籤 {label}")


In [ ]:
import jieba

# 定義停用詞
stopwords = set(['的', '了', '在', '是', '和', '也', '與', '有', '為', '等'])

# 定義分詞並去除停用詞的函數
def tokenize_and_remove_stopwords(text):
    words = jieba.cut(text)  # 使用 jieba 进行分词
    words_filtered = [word for word in words if word not in stopwords]  # 去除停用词
    return ' '.join(words_filtered)

# 將處理後的內容加入 DataFrame 中
df['processed_content'] = df['Content'].apply(tokenize_and_remove_stopwords)

# 新文章
new_article = "台股守穩季線，週線三連紅。（資料照） 〔財經頻道／綜合報導〕美國CPI略高於市場預期，美股漲勢暫歇，本週以來，台股經過兩日大漲後，今（13）日指數震盪走低，終場下跌43.34點，以16782.57點作收，守住季線關卡，成交量為2986.08億元，週線上漲262點，呈現三連紅，緯創失守百元大關，AI族群普遍都是收黑，電子類股以矽光子、網通等次族群比較有表現，傳產輪動到營建、造紙、百貨等接棒演出。 前10大成交額個股漲多跌少，除了AI族群收黑，其他都是紅盤居多，廣達跌12元，收226元，成交額182.32億元，排名第1；台積電終場漲3元，收553元，成交額171.29億元，排名第2；矽統終場漲2.75元，收47.7元，成交額146.25億元，排名第3；定穎投控漲3.3元，收103元，成交額95.92億元，排名第4；緯創跌3.4元，收99.1元，成交額93.89億元，排名第5。 請繼續往下閱讀...  技嘉跌13.5元，收271元，成交額89.05億元，排名第6；創意收1695元平盤，成交額86.78億元，排名第7；聯發科上漲27元，收842元，成交額81.63億元，排名第8；裕隆漲1.1元，收85.1元，成交額66.51億元，排名第9；材料-KY漲 5元，收1185元，成交額63.34億元，排名第10。 一手掌握經濟脈動點我訂閱自由財經Youtube頻道 不用抽 不用搶 現在用APP看新聞 保證天天中獎點我下載APP按我看活動辦法 相關新聞"

# 處理新文章
processed_new_article = tokenize_and_remove_stopwords(new_article)

# 輸出處理後的文章
print(processed_new_article)

# 將新文章轉換為 TF-IDF 表示形式
new_article_tfidf = tfidf_vectorizer.transform([processed_new_article])

# 使用投票分類器進行預測
predicted_label_ensemble = voting_classifier.predict(new_article_tfidf)

# 輸出預測結果
print(f"新文章預測結果: {predicted_label_ensemble}")


In [ ]:
import jieba

# 定義停用詞
stopwords = set(['的', '了', '在', '是', '和', '也', '與', '有', '為', '等'])

# 定義分詞並去除停用詞的函數
def tokenize_and_remove_stopwords(text):
    words = jieba.cut(text)  # 使用 jieba 进行分词
    words_filtered = [word for word in words if word not in stopwords]  # 去除停用词
    return ' '.join(words_filtered)

# 將處理後的內容加入 DataFrame 中
df['processed_content'] = df['Content'].apply(tokenize_and_remove_stopwords)

# 新文章
new_article = "小漲"

# 處理新文章
processed_new_article = tokenize_and_remove_stopwords(new_article)

# 輸出處理後的文章
print(processed_new_article)

# 將新文章轉換為 TF-IDF 表示形式
new_article_tfidf = tfidf_vectorizer.transform([processed_new_article])

# 使用投票分類器進行預測
predicted_label_ensemble = voting_classifier.predict(new_article_tfidf)

# 輸出預測結果
print(f"新文章預測結果: {predicted_label_ensemble}")